# Chapter 3 - Centrality Measures

In this notebook, we explore various centrality measures on a **weighted**, **directed** graph which represents the volume of passengers between US airports in 2008. This dataset is available at: https://www.kaggle.com/flashgordon/usa-airport-dataset#Airports2.csv which is part of the Kaggle Public Datasets: https://www.kaggle.com/datasets

As with the previous notebooks, make sure to set the data directory properly in the next cell.

In [ ]:
datadir = '../Datasets/'

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from statistics import mode
from numpy import inf


In [ ]:
## define colors from pale to dark
colors = ['gainsboro','silver','darkgray','dimgray','black']

## we will use 3 node sizes:
node_sizes  = [6,9,12]

## US Airport Graph

### Volume of Passengers

The nodes are represented by the 3-letter airport codes such as LAX (Los Angeles); 
each line below represents the number of passenges from ```orig_airport``` to ```dest_airport```.
The last column is the volume of passengers that we use as **edge weights**. Ths we will build a weighted,  directed graph.


In [ ]:
## read edges and build weighted directed graph
df = pd.read_csv(datadir+'Airports/connections.csv')
g_airport = ig.Graph.TupleList([tuple(x) for x in df.values], directed=True, edge_attrs=['weight'])
df.head() ## look at a few edges

### Node attributes

We read the node attributes in data frame ```Attr```:
* lat/lon, which we will use as the graph layout
* state (2-letter code)
* city

In [ ]:
## read vertex attributes and add to graph
Attr = pd.read_csv(datadir+'Airports/airports_loc.csv')
## map airports in Attr to the node order in graph g
lookup = {k:v for v,k in enumerate(Attr['airport'])}
l = [lookup[x] for x in g_airport.vs()['name']]
## save lat/lon as tuples for each node:
g_airport.vs()['layout'] = [(Attr['lon'][i],Attr['lat'][i]) for i in l]
g_airport.vs()['state'] = [Attr['state'][i] for i in l]
g_airport.vs()['city'] = [Attr['city'][i] for i in l]
Attr.head() ## first few rows in Attr

In [ ]:
## add a few more attributes for visualization
g_airport.vs()['size'] = node_size[1]
g_airport.vs()['color'] = colors[3]
g_airport.es()['color'] = colors[0]
g_airport.es()['arrow_size'] = 0.33
print('Airport graph:',g_airport.vcount(), 'nodes and', g_airport.ecount(), 'directed edges')

### Check for loops and multiple edges

There are no multiedges (not surprising, edges are weighted here), but there are some loops in the raw data,
for example:
``` 
SEA,SEA,69
```
i.e. same origin and destination airport.

In [ ]:
print('number of loops:',sum(g_airport.is_loop()))
print('number of multi-edges:',sum(g_airport.is_multiple()))

## Connected components

A (sub)graph is **weakly connected** if there is a path between any pair of nodes when we ignore the edge direction (i.e. treat the directed graph as undirected). The airport graph is weakly connected (that is, ignoring directionality) except for 2 airports: DET and WVL that are connected by a single directed edge.

A (sub)graph is **strongly connected** if there is a directed path from each node to every other node. The airport graph is not strongly connected. The largest stongly connected component has size 425.
 

In [ ]:
## count the number of nodes in the giant component (weak connectivity)
print(g_airport.connected_components(mode='weak').giant().vcount(),'out of',
      g_airport.vcount(),'are in giant (weak) component')
print(g_airport.connected_components(mode='strong').giant().vcount(),'out of',
      g_airport.vcount(),'are in giant (strong) component')

In [ ]:
## which two airports are NOT weakly connected to the rest of the graph?
conn_comp = g_airport.connected_components(mode='weak').membership
giant = mode(conn_comp) ## giant component
print('Disconnected airports:')
for i in range(g_airport.vcount()):
    if conn_comp[i] != giant:
        print(g_airport.vs[i]['name'],'has in degree',g_airport.degree(i,mode='in'),
              'and out degree',g_airport.degree(i,mode='out'))   

### Coreness

Looking at coreness (mode = 'all' means that we merge in and out edges, so we consider undirected coreness).
We see a group of nodes with very high coreness: highly connected hub airports (such as 'SFO', 'LAX', 'ATL', etc.).
There are also several nodes with low coreness: peripherial airports.


In [ ]:
g_airport.vs['core'] = g_airport.coreness(mode='all')
plt.hist(g_airport.vs['core'], bins=20, width=3, color=colors[2]);
plt.xlabel('Coreness', fontsize=14)
plt.ylabel('Frequency', fontsize=14);

In [ ]:
## print the airports with maximal coreness:
max_core = np.max(g_airport.vs['core'])
print([v['name'] for v in g_airport.vs if v['core']==max_core])


###  Degree distribution

Below we plot the degree distribution, again with mode='all' (total degree, in and out).
Which airport has maximal degree?

In [ ]:
## degree distribution
g_airport.vs['degree'] = g_airport.degree(mode='all')
plt.hist(g_airport.vs['degree'], bins=20, width=14, color=colors[2])
plt.xlabel('Total degree', fontsize=14)
plt.ylabel('Frequency', fontsize=14);

In [ ]:
## max degree airport
print('Airport with maximal degree:',g_airport.vs[np.argmax(g_airport.vs['degree'])]['name'])

## California Subgraph 

We will look at several **centrality** measures. To speed up the computation and plotting, we consider only the airports in **California**, and the edges within the state.
You can try other states by changing the first line below.


In [ ]:
g_CA = g_airport.subgraph([v for v in g_airport.vs() if v['state'] == 'CA'])


In [ ]:
## Build smaller subgraph 'G' for California
g_CA = g_airport.subgraph([v for v in g_airport.vs() if v['state'] == 'CA'])

## drop isolated vertices (i.e. without in-state connections)
g_CA = g_CA.subgraph([v for v in g_CA.vs() if v.degree()>0])

## remove loops if any
g_CA = g_CA.simplify(multiple=False)
print(g_CA.vcount(),'nodes and',g_CA.ecount(),'directed edges')


In [ ]:
## The graph is weakly connected except for 2 airports
## We color those in red for now
conn_comp = g_CA.connected_components(mode='weak').membership
giant_comp = mode(conn_comp)
g_CA.vs['color'] = colors[2]
print('Nodes outside the giant component:')
for i in range(g_CA.vcount()):
    if conn_comp[i] != giant:
        print(g_CA.vs[i]['name'],'has in degree',g_CA.degree(i,mode='in'),
              'and out degree',g_CA.degree(i,mode='out'))
        g_CA.vs[i]['color'] = colors[4]

In [ ]:
## plot this subgraph using lat/lon as layout
ly = ig.Layout(g_CA.vs['layout'])
## y-axis goes top-down thus the inversion
ly.mirror(1)
ig.plot(g_CA, bbox=(0,0,350,400), layout=ly)

In [ ]:
## same subgraph using a force directed layout
ly = g_CA.layout_fruchterman_reingold()
ig.plot(g_CA, bbox=(0,0,350,400), layout=ly)

## Centrality measures

Most measures defined in Chapter 3 of the book are available directly in ```igraph```.

We compute the following centrality measures for the weighted graph g_CA:
**PageRank**, **Authority** and **Hub**.
For **degree centrality**, we define our own function below and we normalize the weights to get values bounded above by 1. 

For the distance based centrality measures **closeness** and **betweenness**, we do not use the edges weights, so the distance between nodes is the number of hops, and is not based on the number of passengers. This is a natural choice here, since distance between airports (cities) can be viewed as the number of flights needed to travel between those cities.

We compute the above centrality for every node in the g_CA subgraph.

#### Warning for disconnected graphs

The ```igraph``` function to compute **closeness centrality** considers connected components separately. Thus for example the two isolated nodes (with an edge between them) would get maximal value 1 for their centrality. We define our own function to compute closeness centrality as defined in the book, replacing distance (number of hops) with the total number of nodes when no path exists between two nodes.


In [ ]:
## compute the normalized edge weights 
max_weight = np.max(g_CA.es['weight'])
g_CA.es()['normalized_weight'] = [w/max_weight for w in g_CA.es()['weight']]

## directed degree centrality
def degree_centrality(g, weights=None):
    n = g.vcount()
    if g.is_directed():
        dc = [sum(x)/(2*(n-1)) for x in zip(g.strength(mode='in',weights=weights),\
              g.strength(mode='out',weights=weights))]
    else:
        dc = [x/(n-1) for x in g.strength(weights=weights)]
    return dc

In [ ]:
## use distance = number of nodes if disconnected
def closeness_centrality(g):
    n = g.vcount()
    D = np.array(g.distances(mode='all'))
    D[D==inf] = n
    return [(n-1)/sum(D[i]) for i in range(len(D))]    

In [ ]:
## compute several centrality measures for the California subgraph g_CA
df_central = pd.DataFrame({'airport':g_CA.vs()['name'],
                           'degree':degree_centrality(g_CA,weights='normalized_weight'),\
                           'pagerank':g_CA.pagerank(weights='weight'),
                           'authority':g_CA.authority_score(weights='weight'),\
                           'hub':g_CA.hub_score(weights='weight'),'between':g_CA.betweenness(),\
                           'closeness':closeness_centrality(g_CA)})

## normalize the betweenness values
n = g_CA.vcount()
df_central['between'] = [2*x/((n-1)*(n-2)) for x in df_central['between']]

## sort w.r.t. degree centrality, look at top airports
df_central = df_central.sort_values(by='degree', ascending=False)
df_central.head(5)


In [ ]:
## bottom ones
df_central.tail(5)


#### Top airports

The above results agree with intuition in terms of the most central airports in California.
Note however that **SAN** (San Diego) has high values *except* for betweenness, an indication that connecting flights transit mainly via LAX or SFO. 

Below, we plot the California graph again, highlighting the top-3 airports w.r.t. **pagerank**: LAX, SFO, SAN.

In [ ]:
## reset node colours
g_CA.vs()['color'] = colors[2]

## highlight top-3 airports w.r.t. pagerank
g_CA.vs()['pagerank'] = df_central['pagerank']
for x in np.argsort(g_CA.vs()['pagerank'])[-3:]:
    g_CA.vs()[x]['color'] = colors[4]
    g_CA.vs()[x]['size'] = node_sizes[2]
ly = ig.Layout(g_CA.vs['layout'])
ly.mirror(1)
ig.plot(g_CA,bbox=(0,0,350,400),layout=ly)


## Correlation between measures

We use the rank-based **Kendall-tau** correlation to compare the different centrality measures.

We observe high agreement between all measures. In particular, degree-centrality, hub and authority measures are very highly correlated, and so are the distance-based measures (betweenness, closeness).

In [ ]:
## rank-based correlation between measures
df_kt = df_central.corr('kendall', numeric_only=True)
df_kt

## Looking at coreness

We already looked at coreness for the whole airports graph, now we look at the California subgraph, again with mode='all'. Below we show nodes with maximal coreness as larger black dots, and nodes with small coreness as smaller dots.

In [ ]:
## plot nodes w.r.t. coreness
g_CA.vs['color'] = colors[2]
g_CA.vs['size'] = node_sizes[1]
g_CA.vs()['core'] = g_CA.coreness()
max_core = np.max(g_CA.vs()['core'])
min_core = np.min(g_CA.vs()['core'])
for v in g_CA.vs():
    if v['core'] == max_core:
        v['color'] = colors[4]
        v['size'] = node_sizes[2]
    if v['core'] <= min_core+1:
        v['color'] = colors[0]
        v['size'] = node_sizes[0]
ig.plot(g_CA,bbox=(0,0,350,400),layout=ly)

The above uses the geographical layout, so it is not clear what is going on.

Let's use a force directed layout to make the difference between high and low core number clearer. 

The high coreness nodes are clearly seen, and we aso observe the small 2-node connected component that was buried in the previous visualization.


In [ ]:
## Coreness is more clear here
ly = g_CA.layout_kamada_kawai()
ig.plot(g_CA,bbox=(0,0,300,300),layout=ly)

In [ ]:
## vertices with max coreness (13-core) 
## note that there are less than 14 nodes, this is an interesting remark and
## it is because we consider both in and out-going edges by default for directed graph.
V = [v['name'] for v in g_CA.vs() if v['core']==max_core]
print(V)

### Looking at closeness centrality

Using the same layout as above (with high coreness nodes in the middle), we display the closeness centrality scores.

Recall that disconnected node pairs get distance equal to the total number of nodes, so we see very small closeness centrality for the 2 isolated nodes, and the scores for all other scores are influenced by those two nodes, yielding similar values. 

If we zoom in and only consider the connected components, the values for closeness centrality span a wider range, with higher values for high coreness nodes.

In [ ]:
## show closeness centralities, same layout
ix = np.round(closeness_centrality(g_CA),decimals=2)
g_CA.vs['size'] = 3
ig.plot(g_CA,vertex_label=ix,layout=ly,bbox=(0,0,300,300))

In [ ]:
## closeness centrality - looking at the giant component only 
sg = g_CA.connected_components(mode='weak').giant()
ix = np.round(closeness_centrality(sg),decimals=2)
sg.vs['size'] = 3
ig.plot(sg,vertex_label=ix,layout=ly,bbox=(0,0,300,300))


In [ ]:
## compare the above with the igraph function
sg = g_CA.connected_components(mode='weak').giant()
ix = np.round(sg.closeness(),decimals=2)
sg.vs['size'] = 3
ig.plot(sg,vertex_label=ix,layout=ly,bbox=(0,0,300,300))


### comparing coreness with other centrality measures

We add coreness to data frame with centrality measures ```df_central```.
We then group the data in 3 categories: high coreness (value of 13), low (value of 2 or less) or mid-range, and we compute and plot the mean for every other measure.

We see that for all centrality measures except closeness centrality, the values are much higher for nodes with high coreness. The slightly higher pagerank value for 'low' coreness nodes vs 'mid' ones is due to the two airports that are not part of the giant component.

As expected, nodes with small coreness generally have smaller centrality scores. 
This is why for example we can often remove the small core nodes (for example, keeping only the 2-core) to reduce
the size of large graphs without destroying its main structure.


In [ ]:
## group in 3 categories
g_CA.vs()['Core'] = ['low' if v['core']<=2 else 'high' if v['core']==13 else 'mid' for v in g_CA.vs()]
df_central['Coreness'] = g_CA.vs['Core']
df = df_central.groupby('Coreness').mean(numeric_only=True)
df.sort_values(by='degree',inplace=True,ascending=False)
df

In [ ]:
## grouped barplot
bh = np.array(df.loc[['high']])[0]
bm = np.array(df.loc[['mid']])[0]
bl = np.array(df.loc[['low']])[0]
barWidth = 0.25

# Set position of bar on X axis
r1 = np.arange(len(bh))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

# Make the plot
plt.bar(r1, bh, color=colors[4], width=barWidth, edgecolor='white', label='high coreness')
plt.bar(r2, bm, color=colors[2], width=barWidth, edgecolor='white', label='mid coreness')
plt.bar(r3, bl, color=colors[0], width=barWidth, edgecolor='white', label='low coreness')
 
# Add xticks on the middle of the group bars
plt.xlabel('measure',fontsize=14)
plt.xticks([r + barWidth for r in range(len(bh))], df.columns, fontsize=10)
plt.ylabel('score',fontsize=14) 

# Create legend & Show graphic
plt.legend(fontsize=12);


## Delta-centrality example

This is the simple ''pandemic'' spread model as detailed in the book:

*The ''pandemic'' starts at exactly one airport selected uniformly at random from all the airports. Then, the following rules for spreading are applied: (i) in a given airport pandemic lasts only for one round and (ii) in the next round, with probability $\alpha$, the pandemic spreads independently along the flight routes to the destination airports for all connections starting from this airport. Airports can interact with the pandemic many times, and the process either goes on forever or the pandemic eventually dies out. Our goal is to find the expected number of times a given airport interacted with the pandemic, which amounts to the sum over all airports of the expected number of times this airport has the pandemic.*

We use $\alpha$ = 0.1 and plot the (decreasing) delta centrality values in a barplot, using the same 3 colors are with the coreness plot above.

In [ ]:
## Delta-centrality with a simple pandemic spread model
def spread(g, alpha=0.1):
    n = g.vcount()
    I = np.diag(np.repeat(1,n))
    A = np.array(g.get_adjacency().data)
    One = np.ones((n,1))
    X = np.linalg.inv(I-alpha*np.transpose(A))
    Y = np.reshape(X.dot(One)/n,n)
    return np.sum(Y)

def spread_delta_centrality(g, alpha=0.1):
    dc = []
    spr = spread(g, alpha=alpha) # P(G) in the book
    for i in g.vs():
        G = g.copy()
        el = g.incident(i, mode='ALL')
        G.delete_edges(el)
        dc.append((spr-spread(G, alpha=alpha))/spr)
    return dc

In [ ]:
## run the spread model with alpha = 0.1 and show the top airports
g_CA.vs['delta'] = spread_delta_centrality(g_CA, alpha=.1)
df_spread = pd.DataFrame(np.transpose([g_CA.vs['name'],g_CA.vs['delta'],
                                       g_CA.vs['color']]),columns=['airport','delta','color'])
df_spread.sort_values(by='delta', ascending=False, inplace=True)
df_spread.head()

In [ ]:
## plot using the same colors as with coreness plot
heights = [float(x) for x in df_spread['delta']]
bars = df_spread['airport']
y_pos = range(len(bars))
plt.bar(y_pos, heights, color=df_spread['color'] )

# Rotation of the bars names
plt.ylabel('Delta Centrality',fontsize=12)
plt.xticks(y_pos, bars, rotation=90)
plt.yticks();

# legend
color_dict = {'high coreness':colors[4], 'med coreness':colors[2], 'low coreness':colors[0]}         
labels = list(color_dict.keys())
handles = [plt.Rectangle((0,0),1,1, color=color_dict[label]) for label in labels]
plt.legend(handles, labels);


## Group centrality and centralization

We go back to the full airports graph, and we ask the following questions:

* which states have highest delta centralities with respect to efficiency?
* what about centralization for each state subgraph?

Computing efficiency involves the computation of shortest path lengths, which will cause a warning if the graph is disconnected. Warnings can be turned off by un-commenting the next cell.


In [ ]:
## efficiency function given g
def efficiency(g):
    n = g.vcount()
    s = 0
    for i in range(n):
        v = g.get_shortest_paths(i)
        s += np.sum([1/(len(x)-1) for x in v if len(x) > 1])
    return s/(n*(n-1))

## group delta centrality -- we compute for each state
states = list(set(g_airport.vs()['state']))
eff_us = efficiency(g_airport)
dc = []
for s in states:
    v = [x for x in g_airport.vs() if x['state']==s]
    G = g_airport.copy()
    e = []
    for x in v:
        e.extend(g_airport.incident(x, mode='ALL'))
    G.delete_edges(e)
    dc.append((eff_us-efficiency(G))/eff_us)

## sort and show top states
DC = pd.DataFrame({'state':states, 'delta_centrality':dc})
DC = DC.sort_values(by='delta_centrality', ascending=False)
DC.head(5)


In [ ]:
## ... and bottom states
DC.tail(5)


For group centralization, we use the PageRank measure.

In [ ]:
## group centralization (using PageRank) -- by state
## look at states with more than 5 airports only
states = list(set(g_airport.vs()['state']))
page_rank = []
st = []
for s in states:
    v = [x for x in g_airport.vs() if x['state']==s]
    if len(v)>5: ## look at states with more than 5 airports only
        G = g_airport.subgraph(v)
        G = G.simplify(multiple=False) ## drop self-loops
        p = G.pagerank(weights='weight')
        page_rank.append(np.max(p) - np.mean(p))
        st.append(s)

## sort and show top
DC = pd.DataFrame({'State':st, 'Pagerank Centralization':page_rank})
DC = DC.sort_values(by='Pagerank Centralization', ascending=False)
DC.head(5)


We plot the state with highest PageRank centralization (Michigan).

This is a state with one high degree airport (DTW).

In [ ]:
v = [x for x in g_airport.vs() if x['state']=='MI']
G = g_airport.subgraph(v)
G = G.subgraph([v for v in G.vs() if v.degree()>0])
G = G.simplify(multiple=False)
#ig.plot(G, 'central_MI.eps', bbox=(0,0,300,300))
ig.plot(G,bbox=(0,0,300,300))


In [ ]:
## one big hub airport: DTW (Detroit)
G.vs['deg'] = G.degree() # overall degree
for v in G.vs:
    print(v['city'],v['name'],'has degree',v['deg'])

We plot the state with lowest PageRank centralization (ND).

This is a state without high degree (hub) airport.

In [ ]:
## now the bottom  states
DC.tail(5)


In [ ]:
## North Dakota
v = [x for x in g_airport.vs() if x['state']=='ND']
G = g_airport.subgraph(v)
G = G.subgraph([v for v in G.vs() if v.degree()>0])
G = G.simplify(multiple=False)

#ig.plot(G, 'central_ND.eps', bbox=(0,0,300,300))
ig.plot(G, bbox=(0,0,300,300))

In [ ]:
## no big hub city here
G.vs['city']

What should we expect for California? There are hub airports, but several ones. 

In [ ]:
## what about California?
DC[DC['State']=='CA']